# Table of Contents
* [Bokeh Plotting and Custom Tools](#Bokeh-Plotting-and-Custom-Tools)
	* [Building Plots](#Building-Plots)
	* [ColumnDataSource](#ColumnDataSource)
	* [Custom Tools](#Custom-Tools)


# Bokeh Plotting and Custom Tools

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import bokeh.sampledata
bokeh.sampledata.download()

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

## Building Plots

In [ ]:
from bokeh.sampledata.iris import flowers
flowers.head()

In [ ]:
from bokeh.plotting import figure

plot = figure()
plot.circle(flowers['petal_width'],flowers['petal_length'])
show(plot)

While the Scatter chart will automatically color the circle according to the species column, with the plotting interface this operation has to be done manually.

In [ ]:
plot = figure()
    
colors = ['gold','maroon','olivedrab']
for species,color in zip(flowers['species'].unique(),colors):
    idx=flowers['species']==species
    plot.circle(flowers.loc[idx,'petal_width'],flowers.loc[idx,'petal_length'],
                color=color, legend=species, size=10)

plot.legend.location='top_left'
show(plot)

In [ ]:
# add labels
plot.xaxis.axis_label='Petal Width'
plot.yaxis.axis_label='Petal Length'
show(plot)

## ColumnDataSource

In this section I wish to make a plot of the counties in PA and color the county either red or blue based on the results of the 2012 Presidential Election. Blue will be used to color a county where a majority of votes were cast for Barak Obama and red for counties where Mit Romney gained the majority. These data sets are stored in separate CSV files and we must do some Pandas operations to bring the data together. The goal here is to make a single data frame that stores all of the data, including the color value, that I wish to use in my plot.

First, get the latitude and longitude borders for every county in PA

In [ ]:
from bokeh.sampledata.us_counties import data

us_counties = pd.DataFrame(data).T
pennsylvania=us_counties.loc[us_counties['state']=='pa']
pennsylvania.head()

Secondly, I'll load election results from every county in the US and create a data frame of the results from Pennsylvania. This data is in the same order as the previous data frame.

In [ ]:
election_2012 = pd.read_csv('data/2012Election.csv')
pa_results = election_2012.loc[election_2012['state']=='PA',['Obama','Romney']]
pa_results.head()

In [ ]:
# counties were already listed alphabetically in both data sets
pa_results=pa_results.set_index(pennsylvania.index)
pennsylvania=pennsylvania.join(pa_results,how='left')
pennsylvania['Other']=100-(pennsylvania['Romney']+pennsylvania['Obama'])
pennsylvania.head(1)

Now I create a color column that I will use to color each county in my plot.

In [ ]:
def blue_or_red(x):
    if x['Romney'] > x['Obama']:
        return 'red'
    elif x['Obama'] > x['Romney']:
        return 'blue'
    else:
        return 'purple'

In [ ]:
pennsylvania['color'] = pennsylvania.apply(blue_or_red, axis=1)
pennsylvania.head()

In order to integrate deeper with Bokeh I need to make a ColumnDataSource object. There is a direct mapping from a Pandas DataFrame to a Bokeh ColumnDataSource.

In [ ]:
# cast the data frame to ColumnDataSource
from bokeh.charts import ColumnDataSource
source = ColumnDataSource(pennsylvania)

The ColumnDataSource is passed to the plotting function with `source=` and I can now access the columns by strings.

In [ ]:
plot = figure(width=850)
plot.patches('lons', 'lats', color='color', line_color='white', source=source)
show(plot)

## Custom Tools

Now that I have a ColumnDataSource object I can use it's information to prepare a custom hover tool that will show the county name and the percentage of votes cast for each candidate.

I'm going to import all of the tool classes I need to use.

In [ ]:
from bokeh.models import PanTool,WheelZoomTool,ResetTool,HoverTool

In [ ]:
hover = HoverTool(
  tooltips = [
        #dispay_text,column_name
        ('County'   ,'@name'),
        ('% Obama'  ,'@Obama'),
        ('% Romeny' ,'@Romney'),
        ('% Other'  ,'@Other')
    ]
)

tools=[PanTool(), WheelZoomTool(), hover, ResetTool()]

plot = figure(tools=tools, width=850)
plot.patches('lons', 'lats', color='color', line_color='white', source=source)
show(plot)